In [1]:
# Dependencies
import requests
import json
import gmaps
import numpy as np
import pandas as pd

# Google developer API key
from config import gkey

In [2]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [3]:
# import the first csv of retailer data we will use to analyze shop locations
shop_df1 = pd.read_csv("Resources/MedicallyEndorsedRetailers.csv", encoding="utf-8")

# drop any NaN
shop_df1.dropna(how='all')

,License,UBI,Tradename,Privilege,Status,Med Privilege Code,Termination Code,Street Adress,Suite Rm,City,State,County,Zip Code,Date Created,Day Phone,Email
0,415470,6.029980e+15,2020 SOLUTIONS,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,2018 IRON ST STE A,,BELLINGHAM,WA,WHATCOM,982264212,20151012,3603938697,TLOZANO36277@YAHOO.COM
1,413314,6.034150e+15,2020 SOLUTIONS ON THE GUIDE,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,5655 GUIDE MERIDIAN STE A,,BELLINGHAM,WA,WHATCOM,982269722,20151010,3607342020,ACCOUNTING@RETAILBYCHAMPION.COM
2,84154,6.033600e+15,25 TREES,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,3213 S 38TH STE C,,TACOMA,WA,PIERCE,984041094,20180116,2533768562,25treesllc@gmail.com
3,422099,6.035980e+15,365 RECREATIONAL CANNABIS,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,36711 U.S. HIGHWAY 12,,DAYTON,WA,COLUMBIA,983288622,20170926,4259194183,seantherider@hotmail.com
4,423784,6.034300e+15,365 RECREATIONAL CANNABIS,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,17517 15TH AVE NE #B,,SHORELINE,WA,KING,981553801,20160926,4259194183,seantherider@hotmail.com
5,413586,6.033520e+15,4:20 FRIENDLY,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,1515 LEWIS ST,,SPOKANE,WA,SPOKANE,992249785,20151113,5094990999,A420STORE@GMAIL.COM
6,424848,6.035340e+15,420 CAPITOL,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,5980 CAPITOL BLVD SE,,TUMWATER,WA,THURSTON,985015274,20170502,3604026368,Champagneproducts@gmail.com
7,415083,6.033590e+15,420 CARPENTER,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,422 CARPENTER RD STE 105,,LACEY,WA,THURSTON,985037906,20151009,3604026368,CHAMPAGNE232323@YAHOO.COM
8,426678,6.035790e+15,420 ELMA ON MAIN,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,306 W MAIN ST,,ELMA,WA,GRAYS HARBOR,985410000,20180423,3605616972,RTRANMER@HOTMAIL.COM
9,415526,6.033590e+15,420 HOLIDAY,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,2028 10TH AVE,,LONGVIEW,WA,COWLITZ,986324007,20151014,3607033103,DHALAIANDY@GMAIL.COM


In [4]:
# grab only stores where their license was active (issued only)
shop_df1 = shop_df1.loc[shop_df1["Status"] == "ACTIVE (ISSUED)"]

# Use count to see how many shops were removed from original list
# shop_df1.count()

# remove unnecesary shop information 
shop_df1 = shop_df1[["License","Tradename","Street Adress","City","State","County","Zip Code",]]

# rename columns for typo and better clarity
shop_df1 = shop_df1.rename(columns={"Tradename":"Store Name", "Street Adress":"Address"})

# Visualize our first list of shops
shop_df1.head()

,License,Store Name,Address,City,State,County,Zip Code
0,415470,2020 SOLUTIONS,2018 IRON ST STE A,BELLINGHAM,WA,WHATCOM,982264212
1,413314,2020 SOLUTIONS ON THE GUIDE,5655 GUIDE MERIDIAN STE A,BELLINGHAM,WA,WHATCOM,982269722
2,84154,25 TREES,3213 S 38TH STE C,TACOMA,WA,PIERCE,984041094
3,422099,365 RECREATIONAL CANNABIS,36711 U.S. HIGHWAY 12,DAYTON,WA,COLUMBIA,983288622
4,423784,365 RECREATIONAL CANNABIS,17517 15TH AVE NE #B,SHORELINE,WA,KING,981553801


In [5]:
# import the second list of stores we will use to compare shops
shop_df2 = pd.read_csv("Resources/MarijuanaApplicants.csv", encoding="utf-8")

# drop any NaN
shop_df2.dropna(how='all')

# rename License column so we can merge easily
shop_df2 = shop_df2.rename(columns={"License #":"License", "Tradename":"Store Name", "Street Address":"Address", "ZipCode":"Zip Code"})

In [6]:
# filter out stores where license was active (issued only)
shop_df2 = shop_df2.loc[shop_df2["PrivilegeStatus"] == "ACTIVE (ISSUED)"]

# remove unnecesary shop information 
shop_df2 = shop_df2[["License","Store Name","Address","City","State","County","Zip Code"]]

# Visualize our second list
shop_df2.head()

,License,Store Name,Address,City,State,County,Zip Code
0,414884.0,#HASHTAG,3540 STONE WAY N,SEATTLE,WA,KING,981038924.0
1,423413.0,112TH STREET CANNABIS,5809 112TH ST E BLDG B,PUYALLUP,WA,PIERCE,983734323.0
2,415470.0,2020 SOLUTIONS,2018 IRON ST STE A,BELLINGHAM,WA,WHATCOM,982264212.0
3,420908.0,2020 SOLUTIONS,4770 PACIFIC HWY STE A,BELLINGHAM,WA,WHATCOM,982260000.0
5,422239.0,2020 SOLUTIONS EPHRATA,1615 BASIN ST SW,EPHRATA,WA,GRANT,988232134.0


In [7]:
# use outer join to merge two lists and join on store license number
merge_df = pd.merge(shop_df1,shop_df2, on="License", how="outer").fillna("NaN")

merge_df.dropna(how='all')
merge_df.tail()

,License,Store Name_x,Address_x,City_x,State_x,County_x,Zip Code_x,Store Name_y,Address_y,City_y,State_y,County_y,Zip Code_y
460,426849,NaN,NaN,NaN,NaN,NaN,NaN,WESTERN BUD,6186 STATE ROUTE 20 STE B,ANACORTES,WA,SKAGIT,9.82218e+08
461,423993,NaN,NaN,NaN,NaN,NaN,NaN,WHIDBEY ISLAND CANNABIS CO.,1860 SCOTT RD,FREELAND,WA,ISLAND,9.82499e+08
462,353928,NaN,NaN,NaN,NaN,NaN,NaN,YAKIMA WEED CO,1606 FRUITVALE BLVD,YAKIMA,WA,YAKIMA,9.89021e+08
463,422443,NaN,NaN,NaN,NaN,NaN,NaN,YOGAS GREEN TIKI OF BELLINGHAM,20917 NE 72ND AVE STE 102,BATTLE GROUND,WA,CLARK,9.86045e+08
464,415488,NaN,NaN,NaN,NaN,NaN,NaN,ZIA RECREATIONAL,905 SIMPSON AVENUE,HOQUIAM,WA,GRAYS HARBOR,9.85503e+08


In [8]:
# fill in NaN values for _y columns since only missing 6 addresses
for index, row in merge_df.iterrows():
    if row["Store Name_y"] == "NaN":
        merge_df.loc[index, "Store Name_y"] = merge_df.loc[index, "Store Name_x"]
        merge_df.loc[index, "Address_y"] = merge_df.loc[index, "Address_x"]
        merge_df.loc[index, "City_y"] = merge_df.loc[index, "City_x"]
        merge_df.loc[index, "State_y"] = merge_df.loc[index, "State_x"]
        merge_df.loc[index, "County_y"] = merge_df.loc[index, "County_x"]
        merge_df.loc[index, "Zip Code_y"] = merge_df.loc[index, "Zip Code_x"]

In [9]:
# Remove _x columns since we no longer need it
merge_df = merge_df[["License","Store Name_y", "Address_y","City_y","State_y","County_y","Zip Code_y"]]

In [10]:
# Rename columns
shop_df = merge_df.rename(columns={"Store Name_y":"Store Name", 
                        "Address_y":"Address",
                        "City_y":"City",
                        "State_y":"State",
                        "County_y":"County",
                        "Zip Code_y":"Zip Code"})
shop_df.head()

,License,Store Name,Address,City,State,County,Zip Code
0,415470,2020 SOLUTIONS,2018 IRON ST STE A,BELLINGHAM,WA,WHATCOM,982264212
1,413314,2020 SOLUTIONS ON THE GUIDE,5655 GUIDE MERIDIAN STE A,BELLINGHAM,WA,WHATCOM,982269722
2,84154,25 TREES,3213 S 38TH STE C,TACOMA,WA,PIERCE,984041094
3,422099,365 RECREATIONAL CANNABIS,36711 U.S. HIGHWAY 12,DAYTON,WA,COLUMBIA,983288622
4,423784,365 RECREATIONAL CANNABIS,17517 15TH AVE NE #B,SHORELINE,WA,KING,981553801


In [11]:
# Add columns for lat, lng
shop_df["Lat"] = ""
shop_df["Lng"] = ""
shop_df.head()

,License,Store Name,Address,City,State,County,Zip Code,Lat,Lng
0,415470,2020 SOLUTIONS,2018 IRON ST STE A,BELLINGHAM,WA,WHATCOM,982264212,,
1,413314,2020 SOLUTIONS ON THE GUIDE,5655 GUIDE MERIDIAN STE A,BELLINGHAM,WA,WHATCOM,982269722,,
2,84154,25 TREES,3213 S 38TH STE C,TACOMA,WA,PIERCE,984041094,,
3,422099,365 RECREATIONAL CANNABIS,36711 U.S. HIGHWAY 12,DAYTON,WA,COLUMBIA,983288622,,
4,423784,365 RECREATIONAL CANNABIS,17517 15TH AVE NE #B,SHORELINE,WA,KING,981553801,,


In [12]:
# create a params dict that will be updated with address in each iteration
params = {"key": gkey}

# Loop through the shop_df and run a lat/long search for each address
for index,row in shop_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    address = row["Address"]
    city = row["City"]
    state = row["State"]
    
    # update address key value
    params['address'] = f"{address},{city},{state}"
    
    # make request
    shop_lat_lng = requests.get(base_url, params=params)
    
    # print the url
    # print(shop_lat_lng.url)
    
    #convert to json
    shop_lat_lng = shop_lat_lng.json()
    
    # add lat and lng to df
    shop_df.loc[index, "Lat"] = shop_lat_lng["results"][0]["geometry"]["location"]["lat"]
    shop_df.loc[index, "Lng"] = shop_lat_lng["results"][0]["geometry"]["location"]["lng"]

In [13]:
# Visualize to confirm lat and lng
shop_df.tail()

,License,Store Name,Address,City,State,County,Zip Code,Lat,Lng
460,426849,WESTERN BUD,6186 STATE ROUTE 20 STE B,ANACORTES,WA,SKAGIT,982218496,48.4298,-122.607
461,423993,WHIDBEY ISLAND CANNABIS CO.,1860 SCOTT RD,FREELAND,WA,ISLAND,982499430,48.0094,-122.515
462,353928,YAKIMA WEED CO,1606 FRUITVALE BLVD,YAKIMA,WA,YAKIMA,989021237,46.6123,-120.531
463,422443,YOGAS GREEN TIKI OF BELLINGHAM,20917 NE 72ND AVE STE 102,BATTLE GROUND,WA,CLARK,986045227,45.7738,-122.599
464,415488,ZIA RECREATIONAL,905 SIMPSON AVENUE,HOQUIAM,WA,GRAYS HARBOR,985503020,46.9764,-123.882


In [14]:
# Export shop df to a csv 
shop_df.to_csv("Output/Shop_DF.csv")

In [15]:
# Customize the size of the figure
figure_layout = {
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto',
}

# Coordinates for shops
coordinates = shop_df[["Lat","Lng"]].astype(float)

# Plot the map
fig = gmaps.figure(layout=figure_layout)

# Create a layer
store_layer = gmaps.symbol_layer(coordinates, 
                                 fill_color='rgba(0,150,0,0.4)',
                                 stroke_color='rgba(0,0,150,0.4)',
                                scale=2,)

# Add the layer to the map
fig.add_layer(store_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='420px', margin='0 auto 0 auto', padding='1px'))

In [16]:
# Import sorted crime data to plot heatmap of crime data
crime_df = pd.read_csv("Resources/df_crime_sorted_Total.csv")

crime_df.head()

,Unnamed: 0,Year,County,JRN_TOTAL,NIB_TOTAL,Total Population,Total Crime,Crime %
0,22,2012,ADAMS,0,1915,19050,1915,10.05
1,23,2013,ADAMS,159,1971,19200,2130,11.09
2,24,2014,ADAMS,162,1960,19400,2122,10.94
3,25,2015,ADAMS,163,1607,19410,1770,9.12
4,26,2016,ADAMS,127,1398,19510,1525,7.82


In [17]:
# drop unnamed column
crime_df = crime_df.drop(columns="Unnamed: 0")

In [18]:
# create a df based on each year so we can obtain crime % per year
df_2016 = crime_df.loc[crime_df["Year"] == 2016]
df_2015 = crime_df.loc[crime_df["Year"] == 2015]
df_2014 = crime_df.loc[crime_df["Year"] == 2014]
df_2013 = crime_df.loc[crime_df["Year"] == 2013]
df_2012 = crime_df.loc[crime_df["Year"] == 2012]

In [19]:
# Add Lat and Lng columns
df_2016["Lat"] = ""
df_2016["Lng"] = ""

# Reset index
df_2016 = df_2016.reset_index(drop=True)

/Users/Kenneth/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/Kenneth/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
# create a function to get lat and lng from df since we will be looking for counties only
def lat_lng(df):
    
    # create a params dict that will be updated with address in each iteration
    params = {"key": gkey}

    # Loop through the shop_df and run a lat/long search for each address
    for index,row in df.iterrows():
        base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
        county = row["County"]
    
        # update address key value
        params['address'] = f"{county}+COUNTY,Washington"
    
        # make request
        county_lat_lng = requests.get(base_url, params=params)
    
        # print the url
        # print(county_lat_lng.url)
    
        #convert to json
        county_lat_lng = county_lat_lng.json()
    
        # add lat and lng to df
        df.loc[index, "Lat"] = county_lat_lng["results"][0]["geometry"]["location"]["lat"]
        df.loc[index, "Lng"] = county_lat_lng["results"][0]["geometry"]["location"]["lng"]

In [21]:
# Use the function above to obtain lat and lng coordinates for the counties from df_2016
lat_lng(df_2016)

In [22]:
# Visualize df to see coordinates
df_2016.tail()

,Year,County,JRN_TOTAL,NIB_TOTAL,Total Population,Total Crime,Crime %,Lat,Lng
34,2016,WAHKIAKUM,2,137,4000,139,3.48,46.3259,-123.495
35,2016,WALLA WALLA,100,4387,60730,4487,7.39,46.237,-118.585
36,2016,WHATCOM,315,14348,212540,14663,6.90,48.8787,-121.972
37,2016,WHITMAN,53,1385,47940,1438,3.00,46.8363,-117.589
38,2016,YAKIMA,889,19713,250900,20602,8.21,46.5436,-120.756


In [23]:
# Create a crime rate Heatmap layer 2016
fig = gmaps.figure()

# Store 'Lat' and 'Lng' into locations
# We can use the same 'Lat' and 'Lng' coordinates since they are the same counties for each year
locations = df_2016[["Lat","Lng"]].astype(float)

#Convert crime rate % to float and Store
crime_rate = df_2016["Crime %"].astype(float)

heat_layer2016 = gmaps.heatmap_layer(locations, weights=crime_rate,
                                dissipating=True, max_intensity=9,
                                point_radius=0.7)

#Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer2016.dissipating = False
heat_layer2016.max_intensity = 9
heat_layer2016.point_radius = 0.7

fig.add_layer(heat_layer2016)

fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
# Create a crime rate Heatmap layer 2015
fig = gmaps.figure()

#Convert crime rate % to float and Store
crime_rate = df_2015["Crime %"].astype(float)

heat_layer2015 = gmaps.heatmap_layer(locations, weights=crime_rate,
                                dissipating=False, max_intensity=9,
                                point_radius=0.7)

#Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer2015.dissipating = False
heat_layer2015.max_intensity = 9
heat_layer2015.point_radius = 0.7

fig.add_layer(heat_layer2015)

fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
# Create a crime rate Heatmap layer 2014
fig = gmaps.figure()

#Convert crime rate % to float and Store
crime_rate = df_2014["Crime %"].astype(float)

heat_layer2014 = gmaps.heatmap_layer(locations, weights=crime_rate,
                                dissipating=False, max_intensity=9,
                                point_radius=0.7)

#Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer2014.dissipating = False
heat_layer2014.max_intensity = 9
heat_layer2014.point_radius = 0.7

fig.add_layer(heat_layer2014)

fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
# Create a crime rate Heatmap layer 2013
fig = gmaps.figure()

#Convert crime rate % to float and Store
crime_rate = df_2013["Crime %"].astype(float)

heat_layer2013 = gmaps.heatmap_layer(locations, weights=crime_rate,
                                dissipating=False, max_intensity=9,
                                point_radius=0.7)

#Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer2013.dissipating = False
heat_layer2013.max_intensity = 9
heat_layer2013.point_radius = 0.7

fig.add_layer(heat_layer2013)

fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
# Create a crime rate Heatmap layer 2012
fig = gmaps.figure()

#Convert crime rate % to float and Store
crime_rate = df_2012["Crime %"].astype(float)

heat_layer2012 = gmaps.heatmap_layer(locations, weights=crime_rate,
                                dissipating=False, max_intensity=9,
                                point_radius=0.7)

#Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer2012.dissipating = False
heat_layer2012.max_intensity = 9
heat_layer2012.point_radius = 0.7

fig.add_layer(heat_layer2012)

fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# combined map of 2016 heatmap layer and store locations
fig = gmaps.figure()

fig.add_layer(heat_layer2016)
fig.add_layer(store_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
# Import sales to population data to plot heatmap of sales data
sales_df = pd.read_csv("Resources/Sales_to_population.csv")

sales_df.head()

,Unnamed: 0,County,Pop 2015,Pop 2016,Pop 2017,Sales 2015,Sales 2016,Sales 2017,Ratio 2015,Ratio 2016,Ratio 2017
0,0,ASOTIN,22010.0,22150.0,22290.0,78390.0,2390504.0,7901349.03,3.561563,107.923431,354.479544
1,1,BENTON,188590.0,190500.0,193500.0,2840372.0,4970194.0,14867438.10,15.061095,26.090257,76.834305
2,2,CHELAN,75030.0,75910.0,76830.0,1392079.0,4801762.0,7449552.84,18.553632,63.255987,96.961510
3,3,CLALLAM,72650.0,73410.0,74240.0,1400405.0,5217200.0,10671708.82,19.276050,71.069337,143.746078
4,4,CLARK,451820.0,461010.0,471000.0,24732262.0,46119692.0,56832141.35,54.739193,100.040546,120.662720


In [30]:
# Drop unnamed column
sales_df = sales_df.drop(columns=("Unnamed: 0"))

# Add lat and lng columns
sales_df["Lat"] = ""
sales_df["Lng"] = ""

sales_df.head()

,County,Pop 2015,Pop 2016,Pop 2017,Sales 2015,Sales 2016,Sales 2017,Ratio 2015,Ratio 2016,Ratio 2017,Lat,Lng
0,ASOTIN,22010.0,22150.0,22290.0,78390.0,2390504.0,7901349.03,3.561563,107.923431,354.479544,,
1,BENTON,188590.0,190500.0,193500.0,2840372.0,4970194.0,14867438.10,15.061095,26.090257,76.834305,,
2,CHELAN,75030.0,75910.0,76830.0,1392079.0,4801762.0,7449552.84,18.553632,63.255987,96.961510,,
3,CLALLAM,72650.0,73410.0,74240.0,1400405.0,5217200.0,10671708.82,19.276050,71.069337,143.746078,,
4,CLARK,451820.0,461010.0,471000.0,24732262.0,46119692.0,56832141.35,54.739193,100.040546,120.662720,,


In [31]:
# Use our lat_lng function to get latitude and longitude coordinates of each county for sales df
lat_lng(sales_df)

In [32]:
sales_df.head()

,County,Pop 2015,Pop 2016,Pop 2017,Sales 2015,Sales 2016,Sales 2017,Ratio 2015,Ratio 2016,Ratio 2017,Lat,Lng
0,ASOTIN,22010.0,22150.0,22290.0,78390.0,2390504.0,7901349.03,3.561563,107.923431,354.479544,46.146,-117.209
1,BENTON,188590.0,190500.0,193500.0,2840372.0,4970194.0,14867438.10,15.061095,26.090257,76.834305,46.3166,-119.502
2,CHELAN,75030.0,75910.0,76830.0,1392079.0,4801762.0,7449552.84,18.553632,63.255987,96.961510,47.9445,-120.675
3,CLALLAM,72650.0,73410.0,74240.0,1400405.0,5217200.0,10671708.82,19.276050,71.069337,143.746078,48.0405,-124.017
4,CLARK,451820.0,461010.0,471000.0,24732262.0,46119692.0,56832141.35,54.739193,100.040546,120.662720,45.7466,-122.519


In [52]:
# Create a sales Heatmap layer for 2015
fig = gmaps.figure()

# Convert crime rate % to float and Store
sales_rate = sales_df["Ratio 2015"].astype(float) / 5

# County locations
counties = sales_df[["Lat","Lng"]].astype(float)

heat_layer2015s = gmaps.heatmap_layer(counties, weights=sales_rate,
                                dissipating=False, max_intensity=10,
                                point_radius=0.4)

#Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer2015s.dissipating = False
heat_layer2015s.max_intensity = 8
heat_layer2015s.point_radius = .7

fig.add_layer(heat_layer2015s)

fig

Figure(layout=FigureLayout(height='420px'))

In [53]:
# Create a sales Heatmap layer for 2016
fig = gmaps.figure()

# Convert crime rate % to float and Store
sales_rate = sales_df["Ratio 2016"].astype(float) / 5

# County locations
counties = sales_df[["Lat","Lng"]].astype(float)

heat_layer2016s = gmaps.heatmap_layer(counties, weights=sales_rate,
                                dissipating=False, max_intensity=10,
                                point_radius=0.4)

#Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer2016s.dissipating = False
heat_layer2016s.max_intensity = 8
heat_layer2016s.point_radius = 0.7

fig.add_layer(heat_layer2016s)

fig

Figure(layout=FigureLayout(height='420px'))

In [54]:
# Create a sales Heatmap layer for 2017
fig = gmaps.figure()

# Convert crime rate % to float and Store
sales_rate = sales_df["Ratio 2017"].astype(float) / 5

# County locations
counties = sales_df[["Lat","Lng"]].astype(float)

heat_layer2017s = gmaps.heatmap_layer(counties, weights=sales_rate,
                                dissipating=False, max_intensity=10,
                                point_radius=0.4)

#Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer2017s.dissipating = False
heat_layer2017s.max_intensity = 8
heat_layer2017s.point_radius = 0.7

fig.add_layer(heat_layer2017s)

fig

Figure(layout=FigureLayout(height='420px'))

In [55]:
# combined map of 2017 sales heatmap layer and store locations
fig = gmaps.figure()

fig.add_layer(heat_layer2017s)
fig.add_layer(store_layer)

fig

Figure(layout=FigureLayout(height='420px'))